# Lab 04 Informed Search Algorithms

## Task #1: Enhanced Maze Navigation with Multiple Goals


In [ ]:
import math
from queue import PriorityQueue


class MazeNode:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f


def manhattan(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])


def best_first_search(maze, start, goal):
    rows, cols = len(maze), len(maze[0])
    start_node = MazeNode(start)
    start_node.h = manhattan(start, goal)
    start_node.f = start_node.h
    frontier = PriorityQueue()
    frontier.put(start_node)
    visited = set()
    
    while not frontier.empty():
        current_node = frontier.get()
        current_pos = current_node.position
        
        if current_pos == goal:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]
        
        visited.add(current_pos)
        
        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            if (0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and 
                maze[new_pos[0]][new_pos[1]] == 0 and new_pos not in visited):
                neighbor = MazeNode(new_pos, current_node)
                neighbor.h = manhattan(new_pos, goal)
                neighbor.f = neighbor.h  
                frontier.put(neighbor)
                visited.add(new_pos)
    
    return None  


def multi_goal_best_first_search(maze, start, goals):
    remaining_goals = goals.copy()
    overall_path = []
    current_start = start
    
    while remaining_goals:
       
        goal = min(remaining_goals, key=lambda g: manhattan(current_start, g))
        path_segment = best_first_search(maze, current_start, goal)
        if path_segment is None:
            print(f"Goal {goal} not reachable from {current_start}.")
            return None
       
        if overall_path and path_segment[0] == overall_path[-1]:
            overall_path.extend(path_segment[1:])
        else:
            overall_path.extend(path_segment)
        current_start = goal
        remaining_goals.remove(goal)
    return overall_path


maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]
start = (0, 0)
goals = [(4,4), (1,3), (3,1)]
path = multi_goal_best_first_search(maze, start, goals)
if path:
    print("Task 1 - Multi-goal path found:", path)
else:
    print("Task 1 - No path found covering all goals.")


Task 1 - Multi-goal path found: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4), (3, 4), (3, 3), (2, 3), (1, 3), (1, 2), (1, 1), (2, 1), (3, 1)]


## Task #2: Dynamic A* Search with Changing Edge Costs


In [ ]:
import random
import math
import copy
from queue import PriorityQueue


def dynamic_a_star(graph, start, goal, heuristic):
    frontier = [(start, 0 + heuristic[start])]
    g_costs = {start: 0}
    came_from = {start: None}
    visited = set()
    
    while frontier:
        # Simulate a random dynamic update on one edge
        all_edges = [(u, v) for u in graph for v in graph[u]]
        if all_edges:
            u, v = random.choice(all_edges)
            old_cost = graph[u][v]
            new_cost = random.randint(1, 20)
            graph[u][v] = new_cost
            print(f"Traffic update: Edge ({u}->{v}) cost changed from {old_cost} to {new_cost}")
        
        frontier.sort(key=lambda x: x[1])
        current, current_f = frontier.pop(0)
        if current in visited:
            continue
        visited.add(current)
        print(f"Expanding node: {current} with f = {current_f}")
        
        if current == goal:
            path = []
            while current:
                path.append(current)
                current = came_from[current]
            print("Goal reached!")
            return path[::-1]
        
        for neighbor, cost in graph[current].items():
            new_g = g_costs[current] + cost
            f_cost = new_g + heuristic.get(neighbor, math.inf)
            if neighbor not in g_costs or new_g < g_costs[neighbor]:
                g_costs[neighbor] = new_g
                came_from[neighbor] = current
                frontier.append((neighbor, f_cost))
    
    print("Goal not reachable.")
    return None

dynamic_graph = {
    'A': {'B': 4, 'C': 3},
    'B': {'E': 12, 'F': 5},
    'C': {'D': 7, 'E': 10},
    'D': {'E': 2},
    'E': {'G': 5},
    'F': {'G': 16},
    'G': {}
}
heuristic_dyn = {'A': 14, 'B': 12, 'C': 11, 'D': 6, 'E': 4, 'F': 11, 'G': 0}
path_dyn = dynamic_a_star(copy.deepcopy(dynamic_graph), 'A', 'G', heuristic_dyn)
if path_dyn:
    print("Task 2 - Dynamic A* path:", path_dyn)
else:
    print("Task 2 - No path found in dynamic A* search.")


Traffic update: Edge (B->E) cost changed from 12 to 4
Expanding node: A with f = 14
Traffic update: Edge (A->B) cost changed from 4 to 8
Expanding node: C with f = 14
Traffic update: Edge (E->G) cost changed from 5 to 19
Expanding node: B with f = 16
Traffic update: Edge (E->G) cost changed from 19 to 18
Expanding node: E with f = 12
Traffic update: Edge (C->E) cost changed from 10 to 6
Expanding node: D with f = 16
Traffic update: Edge (C->D) cost changed from 7 to 9
Traffic update: Edge (D->E) cost changed from 2 to 9
Expanding node: F with f = 20
Traffic update: Edge (F->G) cost changed from 16 to 3
Expanding node: G with f = 25
Goal reached!
Task 2 - Dynamic A* path: ['A', 'B', 'F', 'G']


## Task #3: Delivery Route Optimization with Time Windows


In [ ]:
import math

def manhattan_distance(p1, p2):
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])

def optimize_delivery_route(deliveries, start, start_time, service_time=1, lateness_penalty=10):
    current_time = start_time
    current_pos = start
    route = []
    remaining = deliveries.copy()
    
    while remaining:
        best_delivery = None
        best_score = math.inf
        best_arrival = None
        for d in remaining:
            travel_time = manhattan_distance(current_pos, d['position'])
            arrival = current_time + travel_time
            if arrival < d['time_window'][0]:
                wait = d['time_window'][0] - arrival
            else:
                wait = 0
            lateness = max(0, arrival - d['time_window'][1])
            score = travel_time + wait + lateness * lateness_penalty
            if score < best_score:
                best_score = score
                best_delivery = d
                best_arrival = arrival
        if best_delivery is None:
            break
        if best_arrival < best_delivery['time_window'][0]:
            current_time = best_delivery['time_window'][0] + service_time
        else:
            current_time = best_arrival + service_time
        current_pos = best_delivery['position']
        route.append(best_delivery['id'])
        remaining.remove(best_delivery)
        print(f"Delivering to {best_delivery['id']} at {best_delivery['position']} (arrival: {best_arrival}, window: {best_delivery['time_window']})")
    return route, current_time

deliveries = [
    {'id': 'A', 'position': (2, 3), 'time_window': (5, 10)},
    {'id': 'B', 'position': (5, 2), 'time_window': (3, 8)},
    {'id': 'C', 'position': (1, 4), 'time_window': (6, 7)},
    {'id': 'D', 'position': (3, 1), 'time_window': (2, 7)}
]
start_location = (0, 0)
start_time = 0
route, finish_time = optimize_delivery_route(deliveries, start_location, start_time)
print("Task 3 - Optimized delivery route:", route)
print("Task 3 - Final time:", finish_time)


Delivering to D at (3, 1) (arrival: 4, window: (2, 7))
Delivering to A at (2, 3) (arrival: 8, window: (5, 10))
Delivering to C at (1, 4) (arrival: 11, window: (6, 7))
Delivering to B at (5, 2) (arrival: 18, window: (3, 8))
Task 3 - Optimized delivery route: ['D', 'A', 'C', 'B']
Task 3 - Final time: 19


## Task #4: A* Search with Real-Time Traffic Updates


In [ ]:
import random
import math
import copy
from queue import PriorityQueue


def update_traffic(graph, update_prob=0.3):
    for u in graph:
        for v in graph[u]:
            if random.random() < update_prob:
                old_cost = graph[u][v]
                change_factor = random.choice([1.5, 2, 0.75, 1])
                new_cost = max(1, int(old_cost * change_factor))
                graph[u][v] = new_cost
                print(f"Traffic update: Edge {u}->{v} cost changed from {old_cost} to {new_cost}")


def a_star_with_traffic(graph, start, goal, heuristic):
    frontier = [(start, 0 + heuristic[start])]
    g_costs = {start: 0}
    came_from = {start: None}
    visited = set()
    
    while frontier:
        update_traffic(graph, update_prob=0.3)
        frontier.sort(key=lambda x: x[1])
        current, current_f = frontier.pop(0)
        if current in visited:
            continue
        visited.add(current)
        print(f"Expanding node: {current} with f = {current_f}")
        
        if current == goal:
            path = []
            while current:
                path.append(current)
                current = came_from[current]
            print("Goal reached!")
            return path[::-1]
        
        for neighbor, cost in graph[current].items():
            new_g = g_costs[current] + cost
            f_cost = new_g + heuristic.get(neighbor, math.inf)
            if neighbor not in g_costs or new_g < g_costs[neighbor]:
                g_costs[neighbor] = new_g
                came_from[neighbor] = current
                frontier.append((neighbor, f_cost))
    print("Goal not reachable.")
    return None

city_graph = {
    'A': {'B': 5, 'C': 7},
    'B': {'D': 3, 'E': 8},
    'C': {'D': 2, 'F': 10},
    'D': {'G': 6},
    'E': {'G': 4},
    'F': {'G': 3},
    'G': {}
}
city_heuristic = {'A': 10, 'B': 8, 'C': 7, 'D': 5, 'E': 4, 'F': 3, 'G': 0}
path_city = a_star_with_traffic(copy.deepcopy(city_graph), 'A', 'G', city_heuristic)
if path_city:
    print("Task 4 - City route found:", path_city)
else:
    print("Task 4 - No route found with traffic updates.")


Traffic update: Edge A->B cost changed from 5 to 5
Traffic update: Edge A->C cost changed from 7 to 5
Traffic update: Edge B->E cost changed from 8 to 6
Traffic update: Edge C->D cost changed from 2 to 4
Traffic update: Edge D->G cost changed from 6 to 12
Expanding node: A with f = 10
Traffic update: Edge A->C cost changed from 5 to 3
Traffic update: Edge B->D cost changed from 3 to 2
Traffic update: Edge C->D cost changed from 4 to 6
Expanding node: C with f = 12
Traffic update: Edge A->C cost changed from 3 to 2
Expanding node: B with f = 13
Traffic update: Edge B->E cost changed from 6 to 9
Traffic update: Edge E->G cost changed from 4 to 6
Expanding node: D with f = 12
Traffic update: Edge A->B cost changed from 5 to 7
Traffic update: Edge B->D cost changed from 2 to 2
Traffic update: Edge D->G cost changed from 12 to 12
Traffic update: Edge E->G cost changed from 6 to 9
Traffic update: Edge F->G cost changed from 3 to 2
Expanding node: E with f = 15
Traffic update: Edge A->C cost 